In [1]:
import torch, torchvision, transformers, datasets
print("✅ All imports working fine!")
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)


✅ All imports working fine!
Torch: 2.9.0+cu128
Transformers: 4.57.1


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="mahwizzzz/UrduBert")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mahwizzzz/UrduBert")
model = AutoModelForMaskedLM.from_pretrained("mahwizzzz/UrduBert")

In [4]:
# =============================================
# 🟢 UrduBERT Fine-Tuning for Sentiment Analysis
# =============================================


# --- Step 1: Download Urdu IMDb Dataset ---
import kagglehub

path = kagglehub.dataset_download("akkefa/imdb-dataset-of-50k-movie-translated-urdu-reviews")
print("📂 Dataset downloaded to:", path)
import pandas as pd

train_path = f"{path}/imdb_urdu_reviews_train.csv"
test_path = f"{path}/imdb_urdu_reviews_test.csv"

# Load both
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Check columns
print("Columns:", train_df.columns.tolist())

# Peek at a few rows
train_df.head()


Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-translated-urdu-reviews' dataset.
📂 Dataset downloaded to: /kaggle/input/imdb-dataset-of-50k-movie-translated-urdu-reviews
Train shape: (40000, 2)
Test shape: (10000, 2)
Columns: ['review', 'sentiment']


,review,sentiment
0,میں نے اسے 80 کی دہائی کے وسط میں ایک کیبل گائ...,positive
1,چونکہ میں نے 80 کی دہائی میں انسپکٹر گیجٹ کارٹ...,negative
2,ایک ایسے معاشرے کی حالت کے بارے میں تعجب کرتا ...,positive
3,مفید البرٹ پیون کی طرف سے ایک اور ردی کی ٹوکری...,negative
4,یہ کولمبو ہے جس کی ہدایتکاری اپنے کیریئر کے اب...,positive


In [5]:
# --- Step 1: Load both CSVs ---
import pandas as pd

train_df = pd.read_csv(path + "/imdb_urdu_reviews_train.csv")
test_df = pd.read_csv(path + "/imdb_urdu_reviews_test.csv")

# Combine them into one dataframe (optional but makes preprocessing easier)
df = pd.concat([train_df, test_df], ignore_index=True)

# --- Step 2: Prepare Data ---
print("Columns:", df.columns.tolist())

# Ensure correct column names — your dataset should have ['review', 'sentiment']
df['labels'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df = df[['review', 'labels']]

# Split into train & test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("✅ Data prepared successfully!")
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Columns: ['review', 'sentiment']
✅ Data prepared successfully!
Train shape: (40000, 2)
Test shape: (10000, 2)


In [6]:
# --- Step 4: Tokenization ---
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def preprocess_function(examples):
    return tokenizer(examples["review"], truncation=True, padding="max_length", max_length=128)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
import importlib
import accelerate
importlib.reload(accelerate)


<module 'accelerate' from '/usr/local/lib/python3.12/dist-packages/accelerate/__init__.py'>

In [8]:
import transformers
import torch
import accelerate
import evaluate

print("Transformers:", transformers.__version__)
print("Torch:", torch.__version__)
print("Accelerate:", accelerate.__version__)


Transformers: 4.57.1
Torch: 2.9.0+cu128
Accelerate: 1.11.0


In [23]:
# --- Step 5: Training UrduBERT ---
from transformers import Trainer
import evaluate

accuracy = evaluate.load("accuracy")

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./urdubert-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # small batch
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)




In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mahwizzzz/UrduBert")

def preprocess_function(examples):
    # Tokenize text
    tokenized = tokenizer(
        examples["review"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # Ensure labels are integers, not lists
    tokenized["labels"] = examples["labels"]
    return tokenized


In [28]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [29]:
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


In [30]:
print(tokenized_train.column_names)
print(tokenized_train[0])


['review', 'labels', '__index_level_0__', 'input_ids', 'attention_mask']
{'labels': tensor(1), 'input_ids': tensor([    0,   367,  9448,  4034,   337, 20357,  4572,  6937,   290,   499,
         3835,   291,  5923,  4070,   304,   295,   617,   525,   332,  4339,
          634,   504,   428,  3706, 17243,  4572,  1271,   322,  5515,   290,
          499,  1280,   341,  3579,   727,   304,  1377,   324,   547,   330,
         2562,   390,  4848,   325,   335,   330,  9098,  2607,  1700,  1001,
          461,   703,  9164,   300,   428, 12711,  7397, 12389,   350,   428,
          425,   290, 13572,  2801,  5254,  2723, 10494,  6052,   350,  3456,
          617,  5816,  4770,   373,  1617,  2764,   290,   485,  8255,   782,
          602,   304,   428,   308,   702, 16679,  1034,   291,  1034,   414,
         1000,   304,   295,  1063,   324,  1034,   330,   836, 20357,  4572,
          300,  3957,  1781,  6409,   322, 10728,   290,   313, 21821,   836,
         4312,   304,   428,   308

In [31]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate

model = AutoModelForSequenceClassification.from_pretrained("mahwizzzz/UrduBert", num_labels=2)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./urdubert-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mahwizzzz/UrduBert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2935317115.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=labels)


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



/tmp/ipython-input-565580499.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.609200,0.455162,0.792600
2,0.411700,0.468814,0.803900
3,0.327800,0.599525,0.805000


TrainOutput(global_step=15000, training_loss=0.4250780975739161, metrics={'train_runtime': 1796.1328, 'train_samples_per_second': 66.81, 'train_steps_per_second': 8.351, 'total_flos': 3974021959680000.0, 'train_loss': 0.4250780975739161, 'epoch': 3.0})

In [33]:
# --- Step 6: Evaluate UrduBERT ---
metrics = trainer.evaluate()
print(metrics)

# --- Step 7: Test UrduBERT on custom sentences ---
text = "یہ فلم بہت شاندار تھی، اداکاری کمال کی تھی۔"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
outputs = model(**inputs)
pred = outputs.logits.argmax(dim=1).item()

if pred == 1:
    print("🌟 Positive review detected!")
else:
    print("💔 Negative review detected!")

{'eval_loss': 0.4551618993282318, 'eval_accuracy': 0.7926, 'eval_runtime': 35.9978, 'eval_samples_per_second': 277.795, 'eval_steps_per_second': 34.724, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

In [34]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1️⃣ Predict on the test dataset
predictions = trainer.predict(tokenized_test)
logits = predictions.predictions
labels = predictions.label_ids

# 2️⃣ Convert logits to predicted labels
preds = logits.argmax(axis=-1)

# 3️⃣ Compute metrics
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

print("✅ Evaluation Summary:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


✅ Evaluation Summary:
Accuracy : 0.7926
Precision: 0.7599
Recall   : 0.8682
F1 Score : 0.8105


In [36]:
from zipfile import ZipFile
import shutil

# Step 1: Zip the saved model folder
shutil.make_archive("urdubert-sentiment-model", 'zip', "./urdubert-sentiment-model")

# Step 2: Download the zip file
from google.colab import files
files.download("urdubert-sentiment-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>